In [ ]:
a = []
while(1):
  a.append(1)

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import zipfile

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/train.zip') as primeira:
  print(*primeira.namelist(), sep='\n')

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/train.zip') as primeira:
  with primeira.open('train.csv') as segunda:
    train = pd.read_csv(segunda, nrows = 10000)

In [ ]:
train

In [ ]:
train.shape

In [ ]:
train.duplicated().sum()

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(train.corr(), annot = True)

In [ ]:
'''
Usando o corr(), nota-se que a maioria dos valores se encontram próximos de 0, representados pelo otm roxo escuro
nesse caso, o heatmap demonstra que os mesmos não possuem correlação
Numa escala, um bom parâmetro para enteder a correção é a seguinte:
ρ = 0,9 a 1 (positivo ou negativo): correlação muito forte;
ρ = 0,7 a 09 (positivo ou negativo): correlação forte;
ρ = 0,5 a 0,7 (positivo ou negativo): correlação moderada;
ρ = 0,3 a 0,5 (positivo ou negativo): correlação fraca;
ρ = 0 a 0,3 (positivo ou negativo): não possui correlação.

Em posse dessas informações registra-se que "dropoff_longitude" tem correlação forte com "pickup_longitude", que significam a longitude que o medidor foi desligado com a
longitude que o medidor foi ligado, o que faz total sentido.
O mesmo se aplica ao dropoff_latitude com pickup_latitude, latitude que o medidor foi desligado com a latitude que o medidor foi ligado, no entanto, esse caso, 
a correlacao é fraca
'''

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/test.zip') as terceira:
  print(*terceira.namelist(), sep='\n')

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/test.zip') as terceira:
  with terceira.open('test.csv') as quarta:
    test = pd.read_csv(quarta)

In [ ]:
test

In [ ]:
test.shape

In [ ]:
test.isnull().sum()

In [ ]:
test.duplicated().sum()

In [ ]:
test.describe()

In [ ]:
test.info()

In [ ]:
#Nota-se que em test, existem 2 colunas a menos: trip_duration (duracao da viagem) e dropoff_datetime (hora que o passageiro saiu do taxi.)
#ambas correlacioandas com a demanda de apurar o tempo de duracao das corridas (variável alvo.)

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(test.corr(), annot = True)

In [ ]:
#o comportamento das variáveis e de seus dependências com as demais, são similares ao de train, onde "dropff_longitude" tem forte correlacao com "pickup_longitude"
# e "dropoff_latitude" com correlacao fraca com "pickup_latitude"
# as demais, não possuem correlacao

# Juntando em um dataframe

In [ ]:
df_completo = train.append(test, ignore_index = True)
df_completo

# Modificações

In [ ]:
#a ideia agora é a de modificar as colunas pickup_datetime e	dropoff_datetime, onde a mesma consta como "Object", sendo que a sua saida corresponde as datas e horarios das corridas
# e para melhor funcionamento da machine learning, preferiu-se arrumar tais informações em colunas distintas.

In [ ]:
#feita essa análise preliminar, onde foi constatado pela inexistencia de valores nulos nas colunas aqui chamade de "features",
#nao existem proedimentos para imputar ou padronizar valores.

In [ ]:
df_completo['pickup_datetime']

In [ ]:
df_completo['Mes Embarque'] = pd.DatetimeIndex(df_completo['pickup_datetime']).month

In [ ]:
df_completo

In [ ]:
df_completo['Ano Embarque'] = pd.DatetimeIndex(df_completo['pickup_datetime']).year

In [ ]:
df_completo['Dia Embarque'] =pd.DatetimeIndex(df_completo['pickup_datetime']).day

In [ ]:
df_completo['Ano Embarque'].unique()

In [ ]:
#pesquisando no stackoverflow, vi que uma maneira de separar a hora da data é transformando a coluna em "datetime"
#e em seguida, usando o "dt.strftime('%H:%M:%S')"
#tentei usar o "dt.strftime" mas é necessário primeiro transformar a coluna em "datetime", caso contrário, dará erro.

In [ ]:
df_completo['pickup_datetime'] = pd.to_datetime(df_completo['pickup_datetime'])

In [ ]:
df_completo['pickup_datetime'] = df_completo['pickup_datetime'].dt.strftime('%H:%M:%S')

In [ ]:
df_completo.head()

Segunda coluna

In [ ]:
df_completo['Mes Desembarque'] = pd.DatetimeIndex(df_completo['dropoff_datetime']).month
df_completo['Ano Desembarque'] = pd.DatetimeIndex(df_completo['dropoff_datetime']).year
df_completo['Dia Desembarque'] = pd.DatetimeIndex(df_completo['dropoff_datetime']).day
df_completo.head()

In [ ]:
df_completo['dropoff_datetime'] = pd.to_datetime(df_completo['dropoff_datetime'])
df_completo.info()

In [ ]:
df_completo['dropoff_datetime'] = df_completo['dropoff_datetime'].dt.strftime("%H:%M:%S")

In [ ]:
df_completo

# Separando as variáveis alvos e features

In [ ]:
train_index = len(train)
train_index

In [ ]:
test_index = len(df_completo) - len(test)
test_index

In [ ]:
train = df_completo[:train_index].copy()
train

In [ ]:
test = df_completo[test_index:].copy()
test

In [ ]:
train

In [ ]:
X = train.drop(['id', 'vendor_id', 'trip_duration'], axis = 1)
y = train['trip_duration']

In [ ]:
X

In [ ]:
y

# Machine Learning

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
!pip install catboost
import catboost
from catboost import CatBoostClassifier, Pool, cv

In [ ]:
from xgboost import XGBRegressor

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size = 0.3)

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
colunas_categoricas = [categoricas for categoricas in X_train.columns if X_train[categoricas].dtype == 'object']

In [ ]:
colunas_categoricas

In [ ]:
colunas_numericas = [numero for numero in X_train.columns if X_train[numero].dtype in ['int64', 'float64']]

In [ ]:
colunas_numericas

In [ ]:
todas_colunas = colunas_categoricas + colunas_numericas
X_train = X_train[todas_colunas].copy()
X_valid = X_valid[todas_colunas].copy()

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputacao = SimpleImputer(strategy='median')

In [ ]:
cat = Pipeline(steps = [
    ('imputacao', SimpleImputer(strategy='most_frequent')),
    ('OneHot', OneHotEncoder(handle_unknown = 'ignore', sparse = False))
])

In [ ]:
preprocessamento = ColumnTransformer(transformers = [
    ('imp', imputacao, colunas_numericas),
    ('cat', cat, colunas_categoricas)
])

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from xgboost import XGBRegressor

In [ ]:
modelo = XGBRegressor(booster = 'gbtree', njobs = -1, verbosity = 0)

In [ ]:
my_pipeline = Pipeline(steps = [
    ('pre', preprocessamento),
    ('mod', modelo)
])

In [ ]:
my_pipeline.fit(X_train, y_train)

In [ ]:
predicao = my_pipeline.predict(X_valid)
predicao

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(y_valid, predicao)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
#validacao = -1 * cross_val_score(my_pipeline, X, y, cv = 3, scoring='neg_mean_absolute_error')

# Test

In [ ]:
test

In [ ]:
#transformar "pickup_datetime" em datetime

In [ ]:
X_test = test[todas_colunas].copy()

In [ ]:
preds = my_pipeline.predict(X_test)
preds